In [16]:
import requests
from urllib.parse import quote

In [17]:
BASE_URL = 'https://test-pred-maint-projects.onrender.com/'

In [18]:
url = 'https://test-pred-maint-projects.onrender.com/list_features'
response = requests.get(url)

# Afficher le statut de la réponse
print('Statut de la réponse:', response.status_code)

# Afficher le contenu de la réponse
print('Contenu de la réponse:', response.text)

Statut de la réponse: 200
Contenu de la réponse: ["Type","Air temperature [K]","Process temperature [K]","Rotational speed [rpm]","Torque [Nm]","Tool wear [min]","Difference temperature [K]","Power"]


In [19]:
response_json = response.json()
response_json

['Type',
 'Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Difference temperature [K]',
 'Power']

In [20]:
class feature_numeric:
    def __init__(self, name:str, min, max):
        self.name = name
        self.min = min
        self.max = max

class feature_list:
    def __init__(self, name:str, values:list):
        self.name = name
        self.values = values

In [45]:
dict_numeric = {}
dict_list = {}

for feature in response_json:
    url2 = BASE_URL+quote(f'features/{feature}')
    response2 = requests.get(url2)
    print(url2)
    print(response2.json())
    if response2.json()['type']=='object':
        dict_list[feature] = feature_list(feature, response2.json()['list'])
    else:
        dict_numeric[feature] = feature_numeric(feature, response2.json()['min'], response2.json()['max'])

https://test-pred-maint-projects.onrender.com/features/Type
{'type': 'object', 'min': 'H', 'max': 'M', 'list': "['M' 'L' 'H']"}
https://test-pred-maint-projects.onrender.com/features/Air%20temperature%20%5BK%5D
{'type': 'float64', 'min': '295.3', 'max': '304.5', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Process%20temperature%20%5BK%5D
{'type': 'float64', 'min': '305.7', 'max': '313.8', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Rotational%20speed%20%5Brpm%5D
{'type': 'int64', 'min': '1168', 'max': '2886', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Torque%20%5BNm%5D
{'type': 'float64', 'min': '3.8', 'max': '76.6', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Tool%20wear%20%5Bmin%5D
{'type': 'int64', 'min': '0', 'max': '253', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Difference%20temperature%20%5BK%5D
{'type': 'float64', 'min': '7.599999999999966', 'max': '12.100000000000025

In [44]:
import ast
print(dict_list['Type'].values.replace(" ", ","))
tmp = eval(dict_list['Type'].values.replace(" ", ","))
print(len(tmp))
print(type(tmp))

['M','L','H']
3
<class 'list'>


In [24]:
len(dict_list)

1

In [ ]:
dict_numeric['Power'].min

In [56]:
url3 = BASE_URL+quote('predict')
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}
data = {
    "Type": "M",
    "Air_temperature": 295.3,
    "Process_temperature": 305.7,
    "Rotational_speed": 1168,
    "Torque": 3.8,
    "Tool_wear": 253
}

response = requests.post(url3, headers=headers,data=json.dumps(data))

# Vérifiez le statut de la réponse
if response.status_code == 200:
    print('Requête réussie.')
else:
    print('Requête échouée.')

# Imprimez le contenu de la réponse
print(url3)
print(response.json())

Requête réussie.
https://test-pred-maint-projects.onrender.com/predict
[{'name': 'Heat dissipation failure', 'proba': 0.9998797178268433}, {'name': 'No failure', 'proba': 5.893323162808883e-09}, {'name': 'Overtrain failure', 'proba': 3.2224573942585266e-07}, {'name': 'Power failure', 'proba': 1.1525643230925198e-06}, {'name': 'Tool wear failure', 'proba': 0.00011871950118802488}]


In [58]:
resp = response.json()

In [63]:
resp[0]['name']


'Heat dissipation failure'

In [64]:
resp[0]['proba']

0.9998797178268433

In [71]:

for i,r in enumerate(resp):
    print(r['name'] + ' : ' + str(round(r['proba']*100, 2)))

Heat dissipation failure : 99.99
No failure : 0.0
Overtrain failure : 0.0
Power failure : 0.0
Tool wear failure : 0.01


In [57]:
type(data)

dict

In [54]:
import requests
import json

#url = 'https://test-pred-maint-projects.onrender.com/predict'
url = BASE_URL+quote('predict')
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}
data = {
    "Type": "M",
    "Air_temperature": 295.3,
    "Process_temperature": 305.7,
    "Rotational_speed": 1168,
    "Torque": 3.8,
    "Tool_wear": 253
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# Vérifiez le statut de la réponse
if response.status_code == 200:
    print('Requête réussie.')
else:
    print('Requête échouée.')

# Imprimez le contenu de la réponse
print(response.text)

Requête réussie.
[{"name":"Heat dissipation failure","proba":0.9998797178268433},{"name":"No failure","proba":5.893323162808883e-09},{"name":"Overtrain failure","proba":3.2224573942585266e-07},{"name":"Power failure","proba":1.1525643230925198e-06},{"name":"Tool wear failure","proba":0.00011871950118802488}]


In [73]:

float_array = [14.46295976638794, 7.287550717592239, 1.7090821638703346, 69.52695846557617, 7.013450562953949]


# conversion en entier avec arrondi
int_array = list(map(round, float_array))

print(int_array)

[14, 7, 2, 70, 7]


In [77]:
import pandas as pd

In [74]:
tmp = [{'item': 'Power >80123.92', 'weight': -25.36146578048767}, {'item': 'Difference temperature [K] >10.23', 'weight': 13.835022513527983}, {'item': 'Rotational speed [rpm] >1454.89', 'weight': -12.05528989944511}, {'item': '160.2< Tool wear [min] <=206.48', 'weight': -11.859791725740928}, {'item': 'Type_M <=0.00', 'weight': -8.13062264145973}]


In [78]:
df = pd.DataFrame(tmp)
df

,item,weight
0,Power >80123.92,-25.361466
1,Difference temperature [K] >10.23,13.835023
2,Rotational speed [rpm] >1454.89,-12.055290
3,160.2< Tool wear [min] <=206.48,-11.859792
4,Type_M <=0.00,-8.130623


In [76]:
for i,r in enumerate(tmp):
    print(r['item'] + ' : ' + str(round(r['weight'],2)))
    

Power >80123.92 : -25.36
Difference temperature [K] >10.23 : 13.84
Rotational speed [rpm] >1454.89 : -12.06
160.2< Tool wear [min] <=206.48 : -11.86
Type_M <=0.00 : -8.13


-------------

In [1]:
import requests
from time import sleep

class ApiRequester:
    def __init__(self, url):
        self.url = url
        self.num_requests_attempts = 0
        self.timeout = 20

    def get_request(self, num_max_requests):
        for _ in range(num_max_requests):
            try:
                response = requests.get(self.url, timeout=self.timeout)
                print(response.status_code)
                if response.status_code == 200:
                    print(response.json())
                    break
                else:
                    self.num_requests_attempts += 1
                    sleep(10)  # Attendre 10 secondes
            except requests.exceptions.Timeout:
                self.num_requests_attempts += 1
                print("La requête a dépassé le délai d'attente.")


In [3]:
BASE_URL = 'https://test-pred-maint-projects.onrender.com/'
# Utilisez la classe avec l'URL de l'API et le nombre de requêtes que vous souhaitez effectuer
api_requester = ApiRequester(BASE_URL)
api_requester.get_request(20)
print(api_requester.num_requests_attempts)

200
hello world !
0
